In [1]:
# imports
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import Dataset
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import EvalPrediction
from transformers import Trainer
from transformers import TrainingArguments
import evaluate
import torch
from typing import Any
from typing import Dict
from typing import Optional
from torch.utils.data import Dataset
from datasets import load_dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load and preprocess News dataset

In [2]:
data = load_dataset('masakhane/masakhanews', 'swa') 

display(data['train'][0])

def change_label(data_point):
    if data_point["label"] == 3:
        data_point["label"] = 1
    else:
        data_point["label"] = 0
    return data_point

politics_label_data = data.map(change_label)

display(politics_label_data)
display(politics_label_data["train"][:5])

{'label': 5,
 'headline': 'Tetesi za soka Ulaya Jumatatu 26.04.2021: Varane, Camara, Nagelsmann, Willock, Azpilicueta',
 'text': 'Chelsea wapo mbele ya  Manchester United na Paris St-Germain katika mbio za kutaka kumsajili beki wa Real Madrid na timu ya taifa ya Ufaransa Raphael Varane, 28. (Mundo Deportivo - in Spanish) Beki wa Guinea Ali Camara, 23, ambaye anachezea klabu ya Young Boys ya Switzerland, amezivutia klabu kadhaa za Ligi ya Primia zikiwemo  Liverpool, Arsenal, Crystal Palace, West Ham United na Norwich. (Team Talk) Bayern Munich imeanzisha mazungumzo ya kutaka kumsajili kocha wa RB Leipzig Julian Nagelsmann. (Independent) Arsenal wapo njia panda juu ya mustakabali wa kiungo wao Joe Willock, 21, ambaye yupo Newcastle kwa mkopo. Arsenal wanahitaji kuuza baadhi ya wachezaji ili kujiimarisha kifedha. (Football London) Kocha wa Atletico Madrid  Diego Simeone ana nia ya kumsajili beki raia wa Uhispania Cesar Azpilicueta, 31, kutoka  Chelsea. (El Gol Digital - in Spanish) Manche

DatasetDict({
    train: Dataset({
        features: ['label', 'headline', 'text', 'headline_text', 'url'],
        num_rows: 1658
    })
    validation: Dataset({
        features: ['label', 'headline', 'text', 'headline_text', 'url'],
        num_rows: 237
    })
    test: Dataset({
        features: ['label', 'headline', 'text', 'headline_text', 'url'],
        num_rows: 476
    })
})

{'label': [0, 0, 1, 0, 0],
 'headline': ['Tetesi za soka Ulaya Jumatatu 26.04.2021: Varane, Camara, Nagelsmann, Willock, Azpilicueta',
  'Je chanjo ya corona ni salama?',
  'Matokeo ya uchaguzi Marekani 2020: Donald Trump amfuta kazi Waziri wa Ulinzi Mark Esper',
  'Je wajua mwanamke na mwanaume hawapaswi kufanya mazoezi pamoja?',
  'Watoto waliolazimika kuwa kimya kuhusu baba zao wakutana na maaskofu jijini Paris'],
 'text': ['Chelsea wapo mbele ya  Manchester United na Paris St-Germain katika mbio za kutaka kumsajili beki wa Real Madrid na timu ya taifa ya Ufaransa Raphael Varane, 28. (Mundo Deportivo - in Spanish) Beki wa Guinea Ali Camara, 23, ambaye anachezea klabu ya Young Boys ya Switzerland, amezivutia klabu kadhaa za Ligi ya Primia zikiwemo  Liverpool, Arsenal, Crystal Palace, West Ham United na Norwich. (Team Talk) Bayern Munich imeanzisha mazungumzo ya kutaka kumsajili kocha wa RB Leipzig Julian Nagelsmann. (Independent) Arsenal wapo njia panda juu ya mustakabali wa kiungo w

## Load Pre-Trained Model
### AfriBerta

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, PreTrainedModel

tokenizer = AutoTokenizer.from_pretrained("castorini/afriberta_base")
model = AutoModelForSequenceClassification.from_pretrained("castorini/afriberta_base")

tokenizer.model_max_length = 512

c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassific

## Tokenize Data

In [4]:

def tokenize_function(datapoints):
    return tokenizer(datapoints["headline_text"], padding="max_length", truncation=True)


tokenized_datasets = politics_label_data.map(tokenize_function, batched=True)

#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

tokenized_datasets['train']

Dataset({
    features: ['label', 'headline', 'text', 'headline_text', 'url', 'input_ids', 'attention_mask'],
    num_rows: 1658
})

## Train Baseline on News Dataset

In [6]:

from transformers import DataCollatorWithPadding



def compute_metrics(pred: EvalPrediction) -> Dict[str, float]:
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer= tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/624 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 0.3625251352787018, 'eval_accuracy': 0.890295358649789, 'eval_f1': 0.8565108047690015, 'eval_precision': 0.8272727272727273, 'eval_recall': 0.9158288770053475, 'eval_runtime': 2.7314, 'eval_samples_per_second': 86.769, 'eval_steps_per_second': 10.983, 'epoch': 1.0}


  0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 0.16360676288604736, 'eval_accuracy': 0.9535864978902954, 'eval_f1': 0.9297777777777778, 'eval_precision': 0.9330221450282241, 'eval_recall': 0.9266310160427808, 'eval_runtime': 2.7933, 'eval_samples_per_second': 84.845, 'eval_steps_per_second': 10.74, 'epoch': 2.0}


...

{'loss': 0.1743, 'grad_norm': 0.05486726388335228, 'learning_rate': 9.935897435897435e-06, 'epoch': 2.4}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


  0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 0.20714546740055084, 'eval_accuracy': 0.9578059071729957, 'eval_f1': 0.9356817194963092, 'eval_precision': 0.9424603174603174, 'eval_recall': 0.9293048128342246, 'eval_runtime': 2.7134, 'eval_samples_per_second': 87.346, 'eval_steps_per_second': 11.056, 'epoch': 3.0}
{'train_runtime': 193.4418, 'train_samples_per_second': 25.713, 'train_steps_per_second': 3.226, 'train_loss': 0.1492373920403994, 'epoch': 3.0}


TrainOutput(global_step=624, training_loss=0.1492373920403994, metrics={'train_runtime': 193.4418, 'train_samples_per_second': 25.713, 'train_steps_per_second': 3.226, 'total_flos': 875500023730176.0, 'train_loss': 0.1492373920403994, 'epoch': 3.0})

In [72]:
import re
from collections import OrderedDict
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
import nltk



def get_top_attended_tokens(input_ids, attention_weights, top_k=5):
    # Sum attention weights across all layers and heads
    aggregated_attentions = attention_weights.sum(dim=(0, 2)).mean(dim=0)

    tokens = tokenizer.convert_ids_to_tokens(input_ids.tolist())
    token_attention_pairs = list(zip(tokens, aggregated_attentions.tolist()))
    
        # Filter out special tokens and punctuation
    filtered_pairs = [
        (token, attn) for token, attn in token_attention_pairs
        if token not in tokenizer.all_special_tokens
        and not re.match(r'[^\w\s]', token)  # Remove punctuation
    ]
    
    
    sorted_pairs = sorted(filtered_pairs, key=lambda x: x[1], reverse=True)
    
    unique_tokens = OrderedDict()
    for token, _ in sorted_pairs:
        if token not in unique_tokens and len(unique_tokens) < top_k:
            unique_tokens[token] = None
    
    return list(unique_tokens.keys())



def get_sentences_with_tokens(full_text, tokens):
    sentences = nltk.sent_tokenize(full_text)
    relevant_sentences = []
    for sentence in sentences:
        highlighted_sentence = sentence
        for token in tokens:
            pattern = re.compile(re.escape(token), re.IGNORECASE)
            highlighted_sentence = pattern.sub(f"**{token}**", highlighted_sentence)
        if highlighted_sentence != sentence:
            relevant_sentences.append(highlighted_sentence)
    return relevant_sentences


# Set the model to evaluation mode
model.eval()

pipe = pipeline(task ="text-classification", model = model, tokenizer=tokenizer, device=device,max_length=512,  truncation=True)
# Disable gradient calculations for inference
with torch.no_grad():
    for example in tokenized_datasets["test"]:
        
        result = pipe(example['headline_text'], truncation=True, max_length=512)
        
        
        #print(result)
        if result[0]['label'] == 'LABEL_1':
        

            # Prepare inputs
            inputs = {
                'input_ids': torch.tensor(example['input_ids']).unsqueeze(0).to(device),
                'attention_mask': torch.tensor(example['attention_mask']).unsqueeze(0).to(device),
            }
            # Handle labels
            if 'label' in example:
                labels = torch.tensor([example['label']]).to(device)
            else:
                labels = None
            # Forward pass
            outputs = model(**inputs, labels=labels, output_attentions=True)
            # Process the outputs
            attentions = outputs.attentions

            # Stack all attention layers
            all_attentions = torch.stack(attentions).squeeze(1)

            # Get top attended tokens

            top_tokens = get_top_attended_tokens(inputs['input_ids'][0], all_attentions, top_k=5)
            print(f"Top 5 attended tokens: {top_tokens}")
            # Get the full text
            full_text = tokenizer.decode(inputs['input_ids'][0])

            # Get sentences containing the top words
            relevant_sentences = get_sentences_with_tokens(full_text, top_tokens)
            
            print("Relevant sentences:")
            for sentence in relevant_sentences:
                print(f"- {sentence}")
            print("\n")
        

Top 5 attended tokens: ['cheza', 'iongoza', 'ambatanish', 'sikika', 'chagua']
Relevant sentences:
- <s> Siku 100 za utawala wa Ruto: Uzito kwa rais wa Kenya ku**ambatanish**a maneno na vitendo Siku zina kasi.Na hazina huruma iwapo ndimi za viongozi wa kisiasa zinatoa cheche na ahadi na ku**sikika** kama wimbo mtamu katika masikio ya wapiga kura na wananchi kwa jumla.
- Kwa rais wa Kenya William Ruto siku zake 100 akiwa usukani ku**iongoza** nchi hiyo zimetimia leo tarehe 22 Disemba 2022.
- Kuna tofauti ya m**cheza** densi na mpiga gita-mmoja ana utaalamu wa kusuka sauti za nyaya za gita kutoa mlio wa kuvutia kuwa muziki.
- Awali ahadi zake zilikuwa rahisi kuzitoa na zili**sikika** kama wimbo kwa wafuasi wake.Lakini sasa,kibarua kimebadilika baada ya muziki kukoma na kazi ngumu ya kutekeleza ahadi hizo kuanza.
- Rais na naibu wake Rigathi Ga**chagua** wamewaambia wananchi kuwa wavumilivu kwani wanahitaji muda zaidi kutimiza hilo.


Top 5 attended tokens: ['orodheshwa', 'idhinishwa', 'te